# Library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from imblearn import over_sampling

# Importing Data

In [ ]:
!gdown --id 1nn7kCgyManm3pz4p4pOnZRDyYBLx2ujo

In [ ]:
df = pd.read_csv("Travel.csv")

# Stage 0

## Data Understanding

### Problem:
Perusahaan 'Trips & Travel.Com' menawarkan lima jenis produk kepada pelanggan mereka, yaitu Paket Basic, Standard, Deluxe, Super Deluxe, dan King. Namun, berdasarkan data tahun lalu, terlihat tingkat konversi yang rendah, dimana hanya 18% dari pelanggan yang membeli paket yang ditawarkan. Perusahaan belum memanfaatkan informasi yang tersedia dari pelanggan dalam menawarkan produk dan menghubungi pelanggan secara  acak, sehingga biaya marketing menjadi tidak efisien.

### Goal:
Dalam usaha ekspansi user base, perusahaan "Trips & Travel.Com" berencana menawarkan produk baru yaitu Wellness Tourism Package. Dalam meluncurkan produk baru, perusahaan ingin:
* Meningkatkan conversion rate dari penawaran produk baru tersebut.
* Membuat biaya marketing menjadi lebih efisien.

### Objectives:
* Membuat model yang dapat memprediksi pelanggan yang berpotensi membeli paket baru.
* Mengidentifikasi segmentasi pelanggan yang berpotensi membeli produk baru berdasarkan atribut mereka.

### Business Metrics:
* Conversion Rate: Persentase dari pelanggan yang ditawarkan yang akhirnya membeli produk tersebut.
* ROI (Return on Investment): Rasio dari nilai keuntungan terhadap biaya marketing.

# Stage 1

## Defining numerical and categorical column

In [ ]:
num_col = [
    "Age",
    "MonthlyIncome",
    "CityTier",
    "NumberOfTrips",
    "NumberOfPersonVisiting",
    "NumberOfChildrenVisiting",
    "DurationOfPitch",
    "NumberOfFollowups",
    "PreferredPropertyStar",
    "PitchSatisfactionScore",
]

cat_col = [
    "Passport",
    "OwnCar",
    "Gender",
    "MaritalStatus",
    "Occupation",
    "Designation",
    "ProductPitched",
]

target = "ProdTaken"

## Descriptive Statistics

In [ ]:
df[num_col].info()

In [ ]:
df[num_col].describe()

In [ ]:
df[cat_col].info()

In [ ]:
df[cat_col].astype(str).describe()

In [ ]:
for col in cat_col:
    uniq = df[col].astype(str).unique().tolist()
    print(f"Kolom {col} memiliki {len(uniq)} unique Value(s): {', '.join(uniq)}")

In [ ]:
df[[target]].astype(str).describe()

In [ ]:
print("Kolom".ljust(26), end="")
print("Jumlah NaN".ljust(23), end="")
print(">20% of total")

for col, nMt, ratio in zip(
    df.columns, df.isna().sum(), df.isna().sum() / len(df) * 100
):
    if nMt != 0:
        print(f"{col}".ljust(30), end="")
        print(f"{nMt}".ljust(25), end="")
        print("Yes") if ratio > 20 else print("No")

Insight:
1. Age (Usia):
    * Terdapat 226 missing values, tidak melampaui 20% dari total data
    * Distribusi data mendekati normal dengan mean di 37.6 dan median di 36
    * Usia termuda ada di umur 18
    * Usia tertua ada di 61
2. Monthly Income (Pendapatan bulanan):
    * Terdapat 233 missing values, tidak melampaui 20% dari total data
    * Distribusi data mendekati normal dengan mean di 23619.9$ dan median di 22347$
    * Terdapat beberapa nilai ekstrim yang dapat dilihat dari nilai min yang jauh dari q1 yaitu 1000$ dan nilai max yang jauh dari q3 yaitu 98678$
3. Number Of Trips (Jumlah rerata trip yang dilakukan customer pertahun):
    * Terdapat 140 missing values, tidak melampaui 20% dari total data
    * Distribusi data mendekati normal dengan mean di 3.2 dan median di 3
    * Terdapat beberapa nilai ekstrim yang dapat dilihat dari nilai max yang jauh dari q3 yaitu 22
4. Number of Person Visiting (Jumlah orang dewasa yang ikut trip):
    * Paling umum customer mengajak 2-3 orang dewasa
    * Terdapat juga yang hanya mengajak 1 orang, yang mungkin adalah orang terdekatnya
    * Paling banyak customer pergi bersama 5 orang dewasa lainnya
5. Number of Children Visiting (Jumlah balita yang ikut trip):
    * Terdapat 66 missing values, tidak melampaui 20% dari total data
    * Paling umum customer membawa 1-2 balita
    * Terdapat customer yang sampai membawa 3 balita, namun juga ada yang tidak sama sekali
6. Duration of Pitch (Durasi promosi):
    * Terdapat 251 missing values, tidak melampaui 20% dari total data
    * Distribusi data mendekati normal dengan mean di 15.5 dan median di 13
    * Terdapat beberapa nilai ekstrim yang dapat dilihat dari nilai max yang jauh dari q3 yaitu 127
7. Number of Follow ups (Jumlah tindak lanjut yang dilakukan setelah dilakukannya promosi):
    * Terdapat 45 missing values, tidak melampaui 20% dari total data
    * Distribusi data mendekati normal dengan mean di 3.7 dan median di 4
    * Terdapat paling banyak 6 kali upaya tindak lanjut dan paling sedikit 1
8. Passport (Apakah customer memiliki passport atau tidak):
    * Sebanyak 71% customer tidak memiliki passport
9. Own Car (Apakah customer memiliki mobil atau tidak):
    * Sebanyak 62% customer memiliki mobil
10. Gender (Jenis kelamin customer):
    * Terdapat 3 gender, bisa jadi ini adalah kesalah tulis (typo)
    * Sebanyak 60% customer berjenis kelamin pria
11. Marital Status (Status pernikahan):
    * Terdapat 4 status berbeda
    * Sebanyak 48% customer berstatus menikah
12. City Tier (Tingkat kelayakan kota yang dihuni customer):
    * Terdapat 3 tingkatan kota
    * Sebanyak 65% customer tinggal di kota tingkat 1
13. Occupation (Tipe pekerjaan yang customer jalani):
    * Terdapat 4 tipe pekerjaan
    * Sebanyak 48% customer bekerja sebagai karyawan
14. Designation (Jabatan pekerjaan dari customer):
    * Terdapat 5 jabatan berbeda
    * Sebanyak 38% customer memiliki jabatan eksekutif
15. Preferred Property Star (Preferensi kelas hotel):
    * Terdapat 26 missing values, tidak melampaui 20% dari total data
    * Normalnya ada di range 1-5 tapi pada dataset hanya berkisar dari 3-5.
    * Sebanyak 61% customer memilih untuk menyewa hotel bintang 3
16. Pitch Satisfaction Score (Tingkat/nilai kepuasan customer terhadap promosi yang diberikan):
    * Terdapat 5 tingkatan berbeda yaitu dari 1-5
    * Sebanyak 30% customer memilih memberi nilai 3
17. Product Pitched (Produk yang dikenalkan pada sesi promosi):
    * Terdapat 5 produk berbeda yang memiliki tingkat yang berbeda
    * Sebanyak 38% customer dikenalkan kepada produk Basic
18. Product Taken (Apakah customer mengambil produk yang dikenalkan atau tidak):
    * Sebanyak 81% tidak mengambil produk yang ditawarkan

## Univariate Analysis

In [ ]:
label_map = {
    "Designation": {
        "Executive": "1",
        "Manager": "2",
        "Senior Manager": "3",
        "AVP": "4",
        "VP": "5",
    },
    "ProductPitched": {
        "Basic": "1",
        "Standard": "2",
        "Deluxe": "3",
        "Super Deluxe": "4",
        "King": "5",
    },
}

In [ ]:
i = 1
plt.figure(figsize=(10, 5 * len(num_col)))
for col in num_col:
    plt.subplot(len(num_col), 2, i)
    sns.histplot(df, x=col, kde=True)
    plt.title(" " * 90 + f"Distribution of {col}", pad=1)
    i += 1
    plt.subplot(len(num_col), 2, i)
    sns.boxplot(df, x=col)
    i += 1
plt.tight_layout()

In [ ]:
def key(col):
    if col not in label_map:
        return None
    else:
        return lambda x: x.map(label_map[col])


i = 1
plt.figure(figsize=(24, 13))
for col in cat_col + [target]:
    plt.subplot(2, 4, i)
    ax = sns.countplot(df.sort_values(by=col, key=key(col)), x=col)
    for p in ax.patches:
        ax.annotate(
            f"{p.get_height()}",
            (p.get_x() + p.get_width() / 2.0, p.get_height()),
            ha="center",
            va="center",
            xytext=(0, 5),
            textcoords="offset points",
        )
    plt.title(f"Distribution of {col}")
    i += 1
plt.show()

# plt.subplot(122)
# tmp = df.sort_values(by=col, key=key(col))[col].value_counts(sort=False) / len(df)
# ax = sns.barplot(x=tmp.index, y=tmp.values)
# plt.xlabel(col)
# for p in ax.patches:
#     ax.annotate(
#         f"{p.get_height()*100:.0f}%",
#         (p.get_x() + p.get_width() / 2.0, p.get_height()),
#         ha="center",
#         va="center",
#         xytext=(0, 5),
#         textcoords="offset points",
#     )

- ProdTaken = 0 (tidak mengambil produk) jauh lebih banyak
- Sebagian besar pelanggan tidak memiliki Passport
- Lebih banyak pelanggan yang memiliki mobil
- Gender yang paling banyak adalah Male. Ada sejumlah kecil pelanggan yang mencantumkan gendernya sebagai "Fe Male"
- Sebagian besar pelanggan sudah menikah. Jumlah pelanggan yang sudah menikah jauh lebih besar dibandinigkan marital status lainnya (unmarried, divorced, dan single)
- Sebagian besar pelanggan adalah karyawan. Untuk pelanggan yang mempunyai bisnis sendiri, lebih banyak pelanggan yang mempunyai bisnis kecil dibandingkan bisnis besar.
- Jabatan yang paling banyak di adalah Executive, diikuti oleh Manager.
- Produk yang paling sering ditawarkan adalah Basic. Produk yang paling jarang ditawarkan adalah King.


## Multivariate Analysis

In [ ]:
# Melihat korelasi antar kolom numerikal

plt.figure(figsize=(12, 7))
sns.heatmap(
    df[num_col + [target]].corr(method="kendall"),
    annot=True,
    cmap="coolwarm",
    fmt=".3f",
)

plt.title("Numerical Columns Correlation")

In [ ]:
# Melihat korelasi antar kolom kategorikal

cat_col_target = cat_col + [target]

p_values_matrix = np.zeros((len(cat_col_target), len(cat_col_target)))
cramers_v_matrix = np.zeros((len(cat_col_target), len(cat_col_target)))

for i in range(len(cat_col_target)):
    for j in range(i + 1, len(cat_col_target)):
        contingency_table = pd.crosstab(df[cat_col_target[i]], df[cat_col_target[j]])
        chi2, p_value, _, _ = chi2_contingency(contingency_table)
        n = contingency_table.sum().sum()
        cramers_v = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))

        p_values_matrix[i, j] = p_value
        p_values_matrix[j, i] = p_value
        cramers_v_matrix[i, j] = cramers_v
        cramers_v_matrix[j, i] = cramers_v

p_values_df = pd.DataFrame(
    p_values_matrix, index=cat_col_target, columns=cat_col_target
)
cramers_v_df = pd.DataFrame(
    cramers_v_matrix, index=cat_col_target, columns=cat_col_target
)

In [ ]:
plt.figure(figsize=(9, 6))
sns.heatmap(
    p_values_df,
    annot=True,
    cmap="coolwarm",
    fmt=".3f",
)

plt.title("Chi-Squared Test - Categorical Correlation")

* Chi2 test menunjukkan adanya korelasi apabila p-value nya <= alpha di mana alpha umumnya menggunakan 0.05
* Target variabel (ProdTaken) memiliki korelasi dengan kolom: Passport, MaritalStatus, Occupation, Designation, ProductPitched

In [ ]:
plt.figure(figsize=(9, 6))
sns.heatmap(
    cramers_v_df,
    annot=True,
    cmap="coolwarm",
    fmt=".3f",
)

plt.title("Cramer's V - Categorical Correlation")

* Nilai Cramer’s V berkisar antara 0 (tidak ada korelasi sama sekali) hingga 1 (sangat kuat/dependan)
* Nilai Cramer’s V lebih menggambarkan dibanding dengan Chi2 test yang sebelumnya
* Dengan menggunakan threshold 0.05 sebagai batas nilai bawah, target variabel (ProdTaken) berkorelasi dengan kolom: Passport, MaritalStatus, Occupation, Designation
* Nilai Cramer’s V kolom Designation dengan ProductPitched menunjukkan adanya korelasi yang sangat tinggi (dependan), mendukung hipotesa awal bahwa company telah menargetkan produk tertentu kepada customer dengan jabatan tertentu
* Passport, Marital Status, Designation, dan ProductPitched mempunyai korelasi yang paling tinggi dengan ProdTaken

In [ ]:
df_temp = df.copy()
df_temp["ProductPitched"] = df_temp["ProductPitched"].map(
    {"Basic": 1, "Standard": 2, "Deluxe": 3, "Super Deluxe": 4, "King": 5}
)
df_temp["Designation"] = df_temp["Designation"].map(
    {"Executive": 1, "Manager": 2, "Senior Manager": 3, "AVP": 4, "VP": 5}
)

In [ ]:
# Melihat seberapa besar korelasi antara pendapatan bulan dengan umur pelanggan

corr = df_temp[["MonthlyIncome", "Age", "ProductPitched"]].corr(method="spearman")
plt.figure(figsize=(9, 6))
sns.heatmap(
    corr,
    annot=True,
    cmap="coolwarm",
    fmt=".3f",
)

plt.title("Age-MonthlyIncome-Product Correlation")

* Pelanggan yang lebih muda cenderung memiliki pendapatan bulanan yang lebih rendah dibanding dengan pelanggan dengan umur yang lebih tua

In [ ]:
# Melihat distribusi pendapatan bulanan dengan jabatan pelanggan

sns.boxplot(
    df_temp,
    y="MonthlyIncome",
    x="Designation",
)

plt.ylim(10000, 50000)

In [ ]:
# Melihat distribusi umur dengan jabatan pelanggan

sns.boxplot(
    df_temp,
    y="Age",
    x="Designation",
)

In [ ]:
# Melihat apakah ada perbedaan pendapatan di beda cakupan perusahaan

dop = (
    df.groupby(["Designation", "Occupation"])["MonthlyIncome"]
    .median()
    .unstack()
    .sort_index(key=key("Designation"))
)
dop.plot(kind="barh")

* Eksekutif merupakan jabatan dengan hirarki terendah bila dilihat dari pendapatan bulanannya, disusul dengan Manager, Senior Manager, AVP, dan VP
* Pelanggan yang paling muda cenderung memiliki jabatan paling rendah, demikian juga memiliki pendapatan bulanan yang paling rendah
* Sebaliknya, pelanggan yang lebih tua cenderung memiliki jabatan yang lebih tinggi, demikian pula memiliki pendapatan bulanan yang lebih tinggi
* Perusahaan telah menargetkan produk yang paling terjangkau (Basic) hanya kepada pelanggan yang paling muda (sekitar 30 tahun), memiliki jabatan paling rendah (Executive), dan memiliki pendapatan bulanan yang paling rendah (sekitar 20.000\$)
* Sama seperti poin sebelumnya, produk produk lainnya juga disesuaikan hingga produk yang paling mewah (King) diberikan hanya kepada pelanggan yang sudah tua (sekitar 50 tahun), memiliki jabatan yang paling tinggi (VP), dan memiliki pendapatan bulanan yang paling tinggi (sekitar 35.000\$)